# Mixing Station Workspace

***************************

Update the .csv file to include the desired constituent electrolytes and their volume ratios, making sure they match up to the correct numbered location on the physical workspace. The columns in the .csv file correspond to the following.. 

| Name | Volume (uL) | Starting Volume (mL) | Density (g/mL) | Aspirate Constant (mbar/uL) | Aspirate Speed (uL/s) |
| --- | --- | --- | --- | --- | --- |

Run the code block below and confirm that your total volume is correct.

*Note: When debugging, you can set `SIM=True` to execute the code without attempting to connect to the gantry or pipette systems.*

In [ ]:
from src.robot_controller import experiment_setup

experiment = experiment_setup.experiment(GANTRY_COM="COM12", PIPETTE_COM="COM11", SIM=True)
experiment.read_csv(CSV_PATH="electrolyte_recipe.csv")

No serial connection to gantry established.
No serial connection to pipette established.
Disc pump gauge pressure set as 1000mbar.
Reading CSV file..


,Name,Volume (uL),Starting Volume (mL),Density (g/mL),Aspirate Constant (mbar/mL),Aspirate Speed (uL/s)
0,O2,50.0,10.0,1.0,0.5,10.0
1,N2,20.0,10.0,1.0,0.5,10.0
2,CH4,200.0,10.0,1.0,0.5,10.0
3,NH4,100.0,10.0,1.0,0.5,10.0
4,CO2,0.0,10.0,1.0,0.5,10.0
5,CH3CH3,70.0,10.0,1.0,0.5,10.0
6,OH,10.0,10.0,1.0,0.5,10.0
7,NaOH,0.0,10.0,1.0,0.5,10.0
8,PO4,25.0,10.0,1.0,0.5,10.0
9,Au,35.0,10.0,1.0,0.5,10.0


Recipe will result in a total electrolyte volume of 0.51mL
Experiment ready to begin: 25/11/2024 15:59:15


## Run Experiment

Running the below code block will begin the electrolyte formulation using the volumes and aspiration variables given in the CSV file. 

*TODO: How will this integrate with an optimiser to update the constituent ratios with each repeat?*

In [ ]:
experiment.run(N=1)

******************************
## Tune Aspiration Variables

Based on the literature, we can expect the aspirate constant to be roughly equal to the system pressure $P_r$ divided by the reservoir volume $V_r$. The system pressure can be assumed equal to atmospheric pressure $\approx$ 1000mbar and the chosen reservoir volume is 2000uL $\implies$ the constant should be about **0.5mbar/uL**.

$$ \Delta P_r = \frac{P_s}{V_r} V_{asp} $$ 

For more viscous liquids, this value may increase based on the ohm's law equivalent of fluid flow through a pipe (the Hagen-Poiseuille equation). To compensate for this, an extra variable is included to slow down the rate of aspiration in an attempt to lower the pressure change required to aspirate more viscous fluids. Ignoring the response time of the pressure PID controller, the aspirate speed will determine how quickly the pressure of the disc pump changes. A typical value would be **10uL/s**.

$$ \Delta P = \frac{8 \mu L}{\pi R^4} Q $$ 

In [ ]:
%matplotlib inline
experiment.tune(name="example", pot_number=1, aspirate_volume=10, starting_volume=50, asp_const_range=[0.4, 0.6], asp_speed_range=[8, 10], N=5)

Tuning of aspiration variables for example: 25/11/2024 15:59:27
Tuning will perform a total of 25 aspirations..
Aspirating using parameters 0.4mbar/mL and 8.0uL/s..
Moving to example..
Pipette pressure set to 1050mbar.
Pipette charged.
Dropping Pipette into example..
Rising to aspiration pressure of 54.0mbar in 1.25s, from charged pressure of 50mbar.
Pipette pressure set to 1050.0mbar.
Pipette pressure set to 1050.211mbar.
Pipette pressure set to 1050.421mbar.
Pipette pressure set to 1050.632mbar.
Pipette pressure set to 1050.842mbar.
Pipette pressure set to 1051.053mbar.
Pipette pressure set to 1051.263mbar.
Pipette pressure set to 1051.474mbar.
Pipette pressure set to 1051.684mbar.
Pipette pressure set to 1051.895mbar.
Pipette pressure set to 1052.105mbar.
Pipette pressure set to 1052.316mbar.
Pipette pressure set to 1052.526mbar.
Pipette pressure set to 1052.737mbar.
Pipette pressure set to 1052.947mbar.
Pipette pressure set to 1053.158mbar.
Pipette pressure set to 1053.368mbar.
Pip

### Update the [CSV](electrolyte_recipe.csv) with the chosen variables before beginning an experiment! 